In [1]:
from pdf2image import convert_from_path

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    images = convert_from_path(pdf)
    for i in range(len(images)):
        images[i].save('supplier_images\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')

In [10]:
import glob
import os

for pdf in glob.glob('Supplier Emission Management\\2022\\*'):
    # print(pdf)
    # print(os.path.split(pdf))
    filename = os.path.split(pdf)[-1].split('.pdf')[0]
    # print(filename)
    pdf_to_images(pdf, filename)

In [6]:
import glob

for img in glob.glob('C:\\air_ticket\\supplier_images\\*'):
    filename = os.path.split(img)[-1].split('.jpg')[0]
    output_filename = 'C:\\air_ticket\\supplier_images\\'+filename+'_output_3'
    command = 'tesseract {} {} -l eng --psm 3'.format(img, output_filename)
    os.system(command)

In [2]:
import re
from dateutil import parser
import datetime

def extract_supplier_name(text):
    supplier_name = ''
    for line in text:
        if 'limited' in line.lower():
            ind = line.lower().split().index('limited')
            supplier_name = ' '.join(line.split()[:ind+1])
            return supplier_name
        elif 'services' in line.lower():
            ind = line.lower().split().index('services')
            supplier_name = ' '.join(line.split()[:ind+1])
            return supplier_name
        elif 'ltd,' in line.lower():
            ind = line.lower().split().index('ltd,')
            supplier_name = ' '.join(line.split()[1:ind+1]).strip(',')
            return supplier_name

def extract_item_name(text):
    item_name = ''
    for i in range(len(text)):
        if 'description' in text[i].lower():
            for j in range(i+1, len(text)):
                if 'no.' in text[j].lower() or 'rate' in text[j].lower() or ('no' in text[j].lower() and len(text[j].split()) == 1):
                    continue
                # print('text[j]---> ', text[j])
                if '|' not in text[j].lower():
                    if 'laptop' in text[j].lower():
                        ind = text[j].lower().split().index('laptop')
                        item_name = ' '.join(text[j].split()[:ind+1])
                        # return item_name
                        # break
                else:
                    item_name = text[j].split('|')[1]

                if ':' in item_name:
                    ind = item_name.index(':')
                    item_name = item_name[:ind]

                return item_name.strip()

def extract_spent_amount(text):
    amount = ''
    for line in text:
        if 'total' in line.lower():
            amount = line.split()[-1]
            if '[' not in amount:
                return amount.strip()

def extract_quantity(text):
    quantity = ''
    flag = 0
    item_name = extract_item_name(text)
    for i in range(len(text)-1):
        if 'quantity' in text[i].lower():
            cols = text[i].split('|')
            # print(cols)
            for j in range(i+1, len(text)):
                if item_name in text[j]:
                    words = text[j].lower().split('|')
                    words = [word.strip() for word in words]
                    # print(words)
                    if len(words) == len(cols) and 'rate' not in words:
                        # print(words)
                        flag = 1
                        quantity = words[3]
                        quantity = re.findall(r'\d+', quantity)[0]
                        return quantity
                    else:
                        break
        if flag == 0:
            if 'total' in text[i].lower():
                words = text[i].split()
                for word in words:
                    if word.isnumeric():
                        quantity = word
                        return quantity

def extract_unit():
    return 'Each'

def extract_spent_date(text):
    # for line in text:
        
    #     day = re.search('\d{2} .* \d{4}|\d{2} .*\d{4}|\d{2}-.*-\d{4}|\d{2}-.*-\d{2}', line)
    #     if day:
    #         try:
    #             print(line)
    #             date = parser.parse(day.group(), fuzzy=True)
    #             print(str(date)[:10])
    #             break
    #         except:
    #             pass
    date = ''
    supplier_name = extract_supplier_name(text)
    for i in range(len(text)):
        if supplier_name in text[i]:
            words = text[i+1].split()
            if '-' in words[-1]:
                date = words[-1]
                formats = ['%d-%b-%Y', '%d-%b-%y']
                for format in formats:
                    try:
                        date = datetime.datetime.strptime(date, format).strftime('%Y-%m-%d')
                    except:
                        pass
                break
            else:
                date = words[-2]+words[-1]
                date = date[:2]+' '+date[2:-4]+' '+date[-4:]
                date = datetime.datetime.strptime(date, '%d %B %Y').strftime('%Y-%m-%d')
                break
    return date

def extract_currency_code(text):
    return 'INR'
            

In [3]:
from prettyprinter import pprint
import glob

for file in glob.glob('C:\\air_ticket\\supplier_comakeit\\ensemble_files\\*'):
    result = {}

    item_name_flag = 0
    spent_amount_flag = 0
    quantity_flag = 0
    unit_of_measure_flag = 0
    spent_date_flag = 0
    supplier_name_flag = 0
    currency_code_flag = 0

    item_name_f = ''
    spent_amount_f = ''
    quantity_f = ''
    unit_of_measure_f = ''
    spent_date_f = ''
    supplier_name_f = ''
    currency_code_f = ''

    with open(file) as fp:
        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            supplier_name = extract_supplier_name(text)
            # print(supplier_name)
            item_name = extract_item_name(text)
            # print(item_name)
            spent_amount = extract_spent_amount(text)
            # print(spent_amount)
            quantity = extract_quantity(text)
            # print(quantity)
            unit_of_measure = extract_unit()
            # print(unit_of_measure)
            spent_date = extract_spent_date(text)
            # print(spent_date)
            currency_code = extract_currency_code(text)
            # print(currency_code)

            text_file = 'interm_output_ensemble_supplier\\{}.txt'.format(filename)
            with open(text_file, 'a', encoding='utf-8') as fp:
                if not item_name:
                    item_name = 'None'
                fp.write(item_name+"\n")
                if not spent_amount:
                    spent_amount = 'None'
                fp.write(spent_amount+"\n")
                if not quantity:
                    quantity = 'None'
                fp.write(quantity+"\n")
                fp.write(unit_of_measure+"\n")
                fp.write(spent_date+"\n")
                if not supplier_name:
                    supplier_name = 'None'
                fp.write(supplier_name+"\n")
                fp.write(currency_code+"\n")
                fp.write('')
                fp.write(str(filename)+"\n") 
                fp.write('----------------------------------------------\n')

            ind = filename.index('page')
            filename = filename[:ind+5]
            final_output_file = 'final_output_ensemble_supplier\\{}.txt'.format(filename)
            with open(final_output_file, 'a') as fp:
                if item_name_flag == 0 and item_name != '':
                    item_name_flag = 1
                    item_name_f = item_name
                    fp.write(item_name_f+"\n")
                if spent_amount_flag == 0 and spent_amount != '':
                    spent_amount_flag = 1
                    spent_amount_f = spent_amount
                    fp.write(spent_amount_f+"\n")
                if quantity_flag == 0 and quantity != '':
                    quantity_flag = 1
                    quantity_f = quantity
                    fp.write(quantity_f+"\n")
                if unit_of_measure_flag == 0 and unit_of_measure != '':
                    unit_of_measure_flag = 1
                    unit_of_measure_f = unit_of_measure
                    fp.write(unit_of_measure_f+"\n")
                if spent_date_flag == 0 and spent_date != '':
                    spent_date_flag = 1
                    spent_date_f = spent_date
                    fp.write(spent_date_f+"\n")
                if supplier_name_flag == 0 and supplier_name != '':
                    supplier_name_flag = 1
                    supplier_name_f = supplier_name
                    fp.write(supplier_name_f+"\n")
                if currency_code_flag == 0 and currency_code != '':
                    currency_code_flag = 1
                    currency_code_f = currency_code
                    fp.write(currency_code_f+"\n")

    result["address"] = None
    result["amount"] = {}
    result["amount"]["amount_paid"] = spent_amount_f
    result["amount"]["currency_code"] = currency_code_f
    result["amount"]["balance_due"] = None
    result["amount"]["due_date"] = None
    result["amount"]["invoice_total"] = None
    result["firm_name"] = None
    result["supplier_name"] = supplier_name_f
    result["invoice_number"] = None
    result["invoice_date"] = spent_date_f
    result["line_items"] = [[{}]]
    result["line_items"][0][0]["amount"] = None
    result["line_items"][0][0]["item_name"] = item_name_f
    result["line_items"][0][0]["s_no"] = 1
    result["line_items"][0][0]["quantity"] = quantity_f
    result["line_items"][0][0]["unit_of_measure"] = unit_of_measure_f
    result["phone"] = None

    pprint([result])

ValueError: time data 'PR IVATELIMI TED)' does not match format '%d %B %Y'

In [217]:
import pandas as pd

path = 'C:\\air_ticket\\final_output_ensemble_supplier\\*'

item_names = []
spent_amounts = []
quantities = []
unit_of_measures = []
spent_dates = []
supplier_names = []

for file in glob.glob(path):
    with open(file) as fp:
        data = fp.read().strip(" \n").split("\n")
        item_names.append(data[0])
        spent_amounts.append(data[1])
        quantities.append(data[2])
        unit_of_measures.append(data[3])
        spent_dates.append(data[4])
        supplier_names.append(data[5])

df = pd.DataFrame({
    "Item Name": item_names,
    "Spent Amount": spent_amounts,
    "Quantity": quantities,
    "Unit of Measure": unit_of_measures,
    "Spent Date": spent_dates,
    "Supplier": supplier_names
})

df.to_excel('supplier_data.xlsx')

In [ ]:
!pip install datefinder

In [153]:
import datefinder

input_string = "Shop No.92, Stearling Phase-1, Trilanga Road, Shahapura| RVE/206 13 JULY2022"
# a generator will be returned by the datefinder module. I'm typecasting it to a list. Please read the note of caution provided at the bottom.
matches = list(datefinder.find_dates(input_string))

if len(matches) > 0:
    # date returned will be a datetime.datetime object. here we are only using the first match.
    date = matches[0]
    print(str(date)[:10])
else:
    print('No dates found')

1992-03-20
